# Columbus 311 API demonstration

The City of Columbus makes its 311 data available via a set of [ArcGIS REST APIs](https://developers.arcgis.com/rest/services-reference/enterprise/get-started-with-the-services-directory.htm).

The service root is available here: [http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer](http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer)

There are two layers available from the service:
  - [All Service Requests - Last 30 Days](http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0)
  - [All Service Requests - Last 3 Years](http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/1)
  
The following code demonstrates how to retrieve data for these two layers in [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) format using the REST APIs.

The API limits the number of records returned to 1000 per request, therefore it is necessary to paginate the results.  The server does not allow pagination by number of records (this functionality exists, but has been disabled), however it is possible to subset the request by time period or geography.  Examples of both are included.

## Import required libraries

Note: Only ```requests``` and ```json``` are required to download the data.  If you only want to download data (and not visualize it), set ```VISUALIZE_DATA = False```.  If you want to display and save the data in CSV format, set ```TABULAR = True```

## Import required libraries

  * If you only want to download data (and not visualize it), set ```VISUALIZE_DATA = False```
  * If you want to display and save the data in CSV format, set ```TABULAR = True```

In [1]:
VISUALIZE_DATA = True
TABULAR = True

import requests
import json
import copy
import pandas as pd
#import pandas as gpd
from datetime import datetime,timedelta

if(VISUALIZE_DATA):
    from ipyleaflet import Map, GeoJSON
if(TABULAR):
    import geopandas as gpd

## All Service Requests - Last 30 Days (no pagination)

Download data and save as a file.

In [2]:
params = {
    "where": "1=1",      # Download all records.  Do not filter by attribute.
    "outFields": "STATUS, SHAPE, OBJECTID, STATUS_DATE, DEPARTMENT_NAME, DIVISION_NAME, SECTION_NAME, DATAHUB_ID, CASE_ID, REPORTED_DATE, REQUEST_CATEGORY, REQUEST_SUBCATEGORY, REQUEST_TYPE, TEAM_NAME, STREET, CITY, ZIP, COLUMBUSCOMMUNITY, AREACOMMISSION, COUNCILDISTRICT, C1PRIORITYAREA, ZIPCODE, LOCATION_ID, LATITUDE, LONGITUDE, REQUEST_MODIFIED",    # Retrieve a subset of the attributes
    "f":"geojson"        # Download in GeoJSON format (other formats are probably not useful)
}    

baseUrl = "http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query"

r = requests.get(url=baseUrl, params=params)
data = r.json()
with open("columbus311_30days.geojson", "w") as f:
    f.write(json.dumps(data, indent=4))
#print(json.dumps(data, indent=4))

In [15]:
from datetime import date

today = date.today()
"columbus311_30days.geojson" + today.strftime("_%m_%d_%Y")

'columbus311_30days.geojson_05_12_2022'

Visualize data as interactive map.

In [4]:
if(VISUALIZE_DATA):
    m = Map(
        center=(40, -83), 
        zoom=10
    )

    geo_json = GeoJSON(
        data=data,
        point_style={
            "radius": 2,
            "color": "blue",
            "fillColor": "blue",
            "fillOpacity": 1
        }
    )
    m.add_layer(geo_json)
else:
    m = "Visualization is disabled"

m

Map(center=[40, -83], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [5]:
import geopandas as gpd
gpd.read_file("columbus311_30days.geojson") 

,STATUS,OBJECTID,STATUS_DATE,DEPARTMENT_NAME,DIVISION_NAME,SECTION_NAME,DATAHUB_ID,CASE_ID,REPORTED_DATE,REQUEST_CATEGORY,...,COLUMBUSCOMMUNITY,AREACOMMISSION,COUNCILDISTRICT,C1PRIORITYAREA,ZIPCODE,LOCATION_ID,LATITUDE,LONGITUDE,REQUEST_MODIFIED,geometry
0,Closed,984321,1.649774e+12,Neighborhoods,311 Service Center,311 Service Center,1353326,CAS-1525541-Z2L3W9,1649746201000,City Staff,...,,,,,,None,NaN,NaN,1649774099000,None
1,Closed,984322,1.649774e+12,City Council,311 Service Center,City Council,1353329,CAS-1525543-M5V6B0,1649747022000,Elected Officials,...,Downtown,,District 7,,43215,1642620,39.966168,-83.008447,1649774177000,POINT (-83.00845 39.96617)
2,Closed,984323,1.649777e+12,Public Service,Street Maintenance,Street Maintenance,1353334,CAS-1525547-B6D4H2,1649747789000,Streets and Traffic,...,Northland,,District 5,,43231,211241,40.065116,-82.932890,1649776632000,POINT (-82.93289 40.06512)
3,Received,984324,NaN,Public Safety,Patrol Operations Subdivision,Patrol Operations Subdivision,1353340,CAS-1525553-V5Y9T1,1649748810000,Public Safety,...,Downtown,,District 7,,43215,1318333,39.965547,-82.993939,1649748827000,POINT (-82.99394 39.96555)
4,Received,984325,NaN,Public Safety,Patrol Operations Subdivision,Patrol Operations Subdivision,1353342,CAS-1525555-B4W0Y8,1649748995000,Public Safety,...,Near East,Near East Area Commission,District 7,Near East,43203,27147,39.976299,-82.958432,1649749012000,POINT (-82.95843 39.97630)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Closed,985556,NaN,Public Service,Refuse Operations (AC),Refuse Operations,1355357,CAS-1527666-Q0S6B7,1649804084000,Refuse,...,South Side,Columbus Southside Area Commission,District 7,Near South,43206,2747,39.944878,-82.976149,1649804105000,POINT (-82.97615 39.94488)
996,Closed,985558,NaN,Public Utilities,Sewers and Drains,Sewers and Drains,1355359,CAS-1527968-R7M0S0,1649804409000,Water,...,Northland,,District 1,,43229,165010,40.103844,-82.969022,1649810689000,POINT (-82.96902 40.10384)
997,Closed,985559,NaN,Public Service,Street Maintenance,Street Maintenance,1355360,CAS-1527984-J4S4K5,1649804414000,Refuse,...,Northland,,District 5,,43231,937129,40.088759,-82.939605,1649804428000,POINT (-82.93961 40.08876)
998,Closed,985560,NaN,Public Service,Refuse Operations,Solid Waste Investigation,1355361,CAS-1527726-G0T8D6,1649804168000,Refuse,...,University District,University Area Commission,District 3,,43202,167379,40.013946,-83.008366,1649804177000,POINT (-83.00837 40.01395)


In [6]:
gdf = gpd.read_file("columbus311_30days.geojson") 
gdf.to_csv("columbus311_30days.csv", index=False)
#gdf["DATE_OPEN"] = gdf["DATE_OPEN"].apply(lambda x:pd.to_datetime(x*1000000))
#gdf["STATUS_DATE"] = gdf["STATUS_DATE"].apply(lambda x:pd.to_datetime(x*1000000))
print("Number of records: {}".format(gdf.shape[0]))
#print("Earliest record: {}".format(gdf["DATE_OPEN"].min()))
#print("Earliest record: {}".format(gdf["DATE_OPEN"].max()))
gdf.head()

Number of records: 1000


,STATUS,OBJECTID,STATUS_DATE,DEPARTMENT_NAME,DIVISION_NAME,SECTION_NAME,DATAHUB_ID,CASE_ID,REPORTED_DATE,REQUEST_CATEGORY,...,COLUMBUSCOMMUNITY,AREACOMMISSION,COUNCILDISTRICT,C1PRIORITYAREA,ZIPCODE,LOCATION_ID,LATITUDE,LONGITUDE,REQUEST_MODIFIED,geometry
0,Closed,984321,1.649774e+12,Neighborhoods,311 Service Center,311 Service Center,1353326,CAS-1525541-Z2L3W9,1649746201000,City Staff,...,,,,,,None,NaN,NaN,1649774099000,None
1,Closed,984322,1.649774e+12,City Council,311 Service Center,City Council,1353329,CAS-1525543-M5V6B0,1649747022000,Elected Officials,...,Downtown,,District 7,,43215,1642620,39.966168,-83.008447,1649774177000,POINT (-83.00845 39.96617)
2,Closed,984323,1.649777e+12,Public Service,Street Maintenance,Street Maintenance,1353334,CAS-1525547-B6D4H2,1649747789000,Streets and Traffic,...,Northland,,District 5,,43231,211241,40.065116,-82.932890,1649776632000,POINT (-82.93289 40.06512)
3,Received,984324,NaN,Public Safety,Patrol Operations Subdivision,Patrol Operations Subdivision,1353340,CAS-1525553-V5Y9T1,1649748810000,Public Safety,...,Downtown,,District 7,,43215,1318333,39.965547,-82.993939,1649748827000,POINT (-82.99394 39.96555)
4,Received,984325,NaN,Public Safety,Patrol Operations Subdivision,Patrol Operations Subdivision,1353342,CAS-1525555-B4W0Y8,1649748995000,Public Safety,...,Near East,Near East Area Commission,District 7,Near East,43203,27147,39.976299,-82.958432,1649749012000,POINT (-82.95843 39.97630)


## All Service Requests - Last 3 Years (no pagination)

Download data and save as a file.

In [2]:
params = {
    "where": "1=1",      # Download all records.  Do not filter by attribute.
    "outFields": "STATUS, SHAPE, OBJECTID, STATUS_DATE, DEPARTMENT_NAME, DIVISION_NAME, SECTION_NAME, DATAHUB_ID, CASE_ID, REPORTED_DATE, REQUEST_CATEGORY, REQUEST_SUBCATEGORY, REQUEST_TYPE, TEAM_NAME, STREET, CITY, ZIP, COLUMBUSCOMMUNITY, AREACOMMISSION, COUNCILDISTRICT, C1PRIORITYAREA, ZIPCODE, LOCATION_ID, LATITUDE, LONGITUDE, REQUEST_MODIFIED",    # Retrieve a subset of the attributes
    "f":"geojson"        # Download in GeoJSON format (other formats are probably not useful)
}    

baseUrl = "http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/1/query"

r = requests.get(url=baseUrl, params=params)
data = r.json()
with open("columbus311_3years.geojson", "w") as f:
    f.write(json.dumps(data, indent=4))
#print(json.dumps(data, indent=4))

Visualize data as interactive map.

In [12]:
if(VISUALIZE_DATA):
    m = Map(
        center=(40, -83), 
        zoom=10
    )

    geo_json = GeoJSON(
        data=data,
        point_style={
            "radius": 2,
            "color": "blue",
            "fillColor": "blue",
            "fillOpacity": 1
        }
    )
    m.add_layer(geo_json)
else:
    m = "Visualization is disabled"

m

Map(center=[40, -83], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [3]:
gdf = gpd.read_file("columbus311_3years.geojson") 
gdf.to_csv("columbus311_3years.csv", index=False)
gdf["DATE_OPEN"] = gdf["DATE_OPEN"].apply(lambda x:pd.to_datetime(x*1000000))
gdf["STATUS_DATE"] = gdf["STATUS_DATE"].apply(lambda x:pd.to_datetime(x*1000000))
print("Number of records: {}".format(gdf.shape[0]))
print("Earliest record: {}".format(gdf["DATE_OPEN"].min()))
print("Earliest record: {}".format(gdf["DATE_OPEN"].max()))
gdf.head()

KeyError: 'DATE_OPEN'

## All Service Requests - Last 3 Years (pagination by time or geography)

First define some runtime parameters.

In [26]:
# Maximum number of records to retrieve from API per request
MAX_RECORDS = 1000

# Incrementally retrieve records between START_DATE and END_DATE (inclusive)
START_DATE = '2022-02-01'
END_DATE = '2022-02-28'

# If True, use spatial segmentation.  If False, use temporal segmentation
SPATIAL = False

# URL for Columbus 311 feature service (All Service Requests - Last 3 Years)
baseUrl = "http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query"

Next let's define some helper functions.

In [36]:
# Get the geographic extents (bounding box) of all requests opened between a start timestamp and end timestamp
def getExtents(start, end):
    params = {
        "where": "DATE_OPEN >= TIMESTAMP '{}' AND DATE_OPEN < TIMESTAMP '{}'".format(start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")),
        "f":"json",
        "returnExtentOnly": True
    }    

    r = requests.get(url=baseUrl, params=params)
    extent = r.json()['extent']
    return extent

# Get the record count of all requests opened between a start timestamp and end timestamp
def getSegmentCount(start, end):
    params = {
        "where": "DATE_OPEN >= TIMESTAMP '{}' AND DATE_OPEN < TIMESTAMP '{}'".format(start.strftime("%Y-%m-%d %H:%M:%S"), end.strftime("%Y-%m-%d %H:%M:%S")),
        "f":"json",
        "returnCountOnly": True,
    }    

    r = requests.get(url=baseUrl, params=params)
    return int(r.json()['count'])

# Get the spatial and attribute data for all requests opened between a start timestamp and end timestamp
# NOTE: This will return a maximum of 1000 records.  You must ensure that the start and end times are selected
# such that the record count is no greater than 1000.
# Output is an array of GeoJSON features
def getSegmentData(start, end):
    params = {
        "where": "DATE_OPEN >= TIMESTAMP '{}' AND DATE_OPEN < TIMESTAMP '{}'".format(start.strftime("%Y-%m-%d %H:%M:%S"), end.strftime("%Y-%m-%d %H:%M:%S")),
        "f":"geojson",
        "outFields": "OBJECTID,SWR_TYPE,SR_CATEGORY",
    } 

    r = requests.get(url=baseUrl, params=params)
    return r.json()['features']

# Get the record count of all requests opened between a start timestamp and end timestamp AND within the specified
# geographic bounding box.
# bbox = [xmin, ymin, xmax, ymax]
# Bounding box coordinates are specified in the default coordinate reference system for the feature service,
# namely EPSG:3857 at the time of writing.
def getSegmentCountSpatial(start, end, bbox):
    params = {
        "where": "DATE_OPEN >= TIMESTAMP '{}' AND DATE_OPEN < TIMESTAMP '{}'".format(start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")),
        "f":"json",
        "returnCountOnly": True,
        "geometry": "{},{},{},{}".format(bbox[0],bbox[1],bbox[2],bbox[3]),
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects"
    }    

    r = requests.get(url=baseUrl, params=params)
    return int(r.json()['count'])

# Get the spatial and attribute data for all requests opened between a start timestamp and end timestamp AND within
# the specified geographic bounding box. 
# NOTE: This will return a maximum of 1000 records.  You must ensure that the start and end times are selected
# such that the record count is no greater than 1000.
# bbox = [xmin, ymin, xmax, ymax]
# Bounding box coordinates are specified in the default coordinate reference system for the feature service,
# namely EPSG:3857 at the time of writing.
# Output is an array of GeoJSON features
def getSegmentDataSpatial(start, end, bbox):
    params = {
        "where": "DATE_OPEN >= TIMESTAMP '{}' AND DATE_OPEN < TIMESTAMP '{}'".format(start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")),
        "f":"geojson",
        "outFields":"*",
        "geometry": "{},{},{},{}".format(bbox[0],bbox[1],bbox[2],bbox[3]),
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects"
    }    

    r = requests.get(url=baseUrl, params=params)
    
    return r.json()['features']

The following functions provide loops that decompose the time range or geographic bounding box into increasingly smaller slices until the all slices contain fewer than MAX_RECORDS, then iteratively retrieve the records in each slice and assemble them into a single feature array. Final array may contain duplicates due to boundary conditions. These will be removed in the next step.

In [33]:
# Spatial slices
def getIntervalFeaturesSpatial(start, end):

    extents = getExtents(start, end)

    # Start with a single segment that spans the entire x-range of the bounding box
    segments=[[extents['xmin'],extents['xmax']]]

    bbox = [extents['xmin'], extents['ymin'], extents['xmax'], extents['ymax']]

    # Continue to decompose the span into segments until the no segment has more than MAX_RECORDS
    iterations = 0
    maxExceeded=True
    while(maxExceeded):
        iterations += 1
        segmentCount = []
        maxExceeded=False

        # For each segment, get the record count within the segment
        for i in range(0, len(segments)):
            bbox = [segments[i][0], extents['ymin'], segments[i][1], extents['ymax']]
            segmentCount.append(getSegmentCountSpatial(start, end, bbox))
            #print(iterations, i, segmentCount[i], segments[i][0], segments[i][1])

        # Check whether the record count in any segment exceeds MAX_RECORDS.  If it does,
        # divide that segment into two slices by creating a new segment boundary at half
        # the x distance
        for i in range(0, len(segments)):
            if(segmentCount[i] > MAX_RECORDS):
                maxExceeded=True
                segments.insert(i+1, [(segments[i][1]+segments[i][0])/2 , segments[i][1]])
                segments[i][1] = segments[i+1][0]


    # Test for duplicate values
    #totalCount = getSegmentCountSpatial(start, start, bbox) 
    #segmentCountSeries= pd.Series(segmentCount)
    #iterativeCount = segmentCountSeries.sum()
    #if(iterativeCount != totalCount):
    #    print("WARNING: Feature count is {}.  Expected {}. Check for duplicates.".format(iterativeCount, totalCount))

    # Fetch the data for each segment and assemble into a single array
    features=[]
    for i in range(0, len(segments)):
        if segmentCount[i] != 0:
            bbox = [segments[i][0], extents['ymin'], segments[i][1], extents['ymax']]
            features += getSegmentDataSpatial(start, end, bbox)

    return features

# Temporal slices
def getIntervalFeatures(start, end):

     # Start with a single segment that spans the entire time range
    segments=[[start,end]]

    # Continue to decompose the span into segments until the no segment has more than MAX_RECORDS
    iterations = 0
    maxExceeded=True
    while(maxExceeded):
        iterations += 1
        segmentCount = []
        maxExceeded=False

        # For each segment, get the record count within the segment
        for i in range(0, len(segments)):
            segmentCount.append(getSegmentCount(segments[i][0], segments[i][1]))
            #print(iterations, i, segmentCount[i], segments[i][0], segments[i][1])

        # Check whether the record count in any segment exceeds MAX_RECORDS.  If it does,
        # divide that segment into two slices by creating a new segment boundary at half of 
        # the time span
        for i in range(0, len(segments)):
            if(segmentCount[i] > MAX_RECORDS):
                maxExceeded=True
                segments.insert(i+1, [segments[i][0]+(segments[i][1]-segments[i][0])/2 , segments[i][1]])
                segments[i][1] = segments[i+1][0]

    # Test for duplicate values
    #totalCount = getSegmentCount(start, end) 
    #segmentCountSeries= pd.Series(segmentCount)
    #iterativeCount = segmentCountSeries.sum()
    #if(iterativeCount != totalCount):
    #    print("WARNING: Feature count is {}.  Expected {}. Check for duplicates.".format(iterativeCount, totalCount))

    # Fetch the data for each segment and assemble into a single array
    features=[]
    for i in range(0, len(segments)):
        if segmentCount[i] != 0:
            features += getSegmentData(segments[i][0], segments[i][1])

    return features

Now run the loop!

In [34]:
startDateTimestamp = datetime.strptime(START_DATE, "%Y-%m-%d")
endDateTimestamp = datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)  # Increment by 1 day, so that we can use "less than" query
nDays = (endDateTimestamp-startDateTimestamp).days

features = []
for i in range(0, nDays):
    todayTimestamp = startDateTimestamp+timedelta(days=i)
    tomorrowTimestamp = startDateTimestamp + timedelta(days=i+1)
    print("Getting data for {}".format(todayTimestamp.strftime("%Y-%m-%d")))
    if SPATIAL:
        features += getIntervalFeaturesSpatial(todayTimestamp, tomorrowTimestamp)
    else:
        features += getIntervalFeatures(todayTimestamp, tomorrowTimestamp)

Getting data for 2022-02-01
http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-01+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-02+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 87, 'geometry': {'type': 'Point', 'coordinates': [-82.94252671301213, 40.00130182178123]}, 'properties': {'OBJECTID': 87, 'SWR_TYPE': 'BED BUG ITEMS/SPECIAL COLLECTION', 'SR_CATEGORY': 'PUBLIC HEALTH'}}, {'type': 'Feature', 'id': 115, 'geometry': {'type': 'Point', 'coordinates': [-82.98321751807889, 40.12678400822214]}, 'properties': {'OBJECTID': 115, 'SWR_TYPE': 'REPLACE DAMAGED 90-GALLON REFUSE CONTAINER', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 116, 'geometry': {'type': 'Point', 'coordinates': [-82.98321751807889, 40.12678400822214]}

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-02+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-03+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 9, 'geometry': {'type': 'Point', 'coordinates': [-83.02089640674528, 40.108155786312096]}, 'properties': {'OBJECTID': 9, 'SWR_TYPE': 'INSPECT RESTAURANT/FOOD ESTABLISHMENT', 'SR_CATEGORY': 'PUBLIC HEALTH'}}, {'type': 'Feature', 'id': 89, 'geometry': {'type': 'Point', 'coordinates': [-83.02347922578218, 39.953691874339434]}, 'properties': {'OBJECTID': 89, 'SWR_TYPE': 'SWI/ILLEGAL MATERIALS IN THE ALLEY', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 215, 'geometry': {'type': 'Point', 'coordinates': [-83.03110620461536, 39.951885798293546]}, 'properties': {'OBJECTID': 215

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-03+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-04+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 65, 'geometry': {'type': 'Point', 'coordinates': [-83.07724685225395, 39.946784681962704]}, 'properties': {'OBJECTID': 65, 'SWR_TYPE': 'REPAIR OF STREET LIGHTING', 'SR_CATEGORY': 'STREET LIGHTING'}}, {'type': 'Feature', 'id': 183, 'geometry': {'type': 'Point', 'coordinates': [-83.07539568496482, 39.9454870513091]}, 'properties': {'OBJECTID': 183, 'SWR_TYPE': 'REPAIR OF STREET LIGHTING', 'SR_CATEGORY': 'STREET LIGHTING'}}, {'type': 'Feature', 'id': 186, 'geometry': {'type': 'Point', 'coordinates': [-83.04512183915413, 39.94773720786802]}, 'properties': {'OBJECTID': 186, 'SWR_TYPE': 'REPAIR OF S

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-04+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-05+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 2, 'geometry': {'type': 'Point', 'coordinates': [-83.08089773043446, 40.0240624512559]}, 'properties': {'OBJECTID': 2, 'SWR_TYPE': 'WATER LINE BREAK', 'SR_CATEGORY': 'WATER, SEWERS & DRAINS'}}, {'type': 'Feature', 'id': 100, 'geometry': {'type': 'Point', 'coordinates': [-82.84442287403319, 39.91223221873392]}, 'properties': {'OBJECTID': 100, 'SWR_TYPE': 'REPLACE MISSING/STOLEN 90-GALLON CONTAINER', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 50, 'geometry': {'type': 'Point', 'coordinates': [-83.00382460838851, 40.15058693328414]}, 'properties': {'OBJECTID': 50, 'SWR_T

Getting data for 2022-02-07
http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-07+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-07+12%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 75, 'geometry': {'type': 'Point', 'coordinates': [-82.97853308145162, 40.038735948193164]}, 'properties': {'OBJECTID': 75, 'SWR_TYPE': '64-GALLON MISSED SERVICE', 'SR_CATEGORY': 'RECYCLING / YARD WASTE'}}, {'type': 'Feature', 'id': 79, 'geometry': {'type': 'Point', 'coordinates': [-82.98212215913381, 40.13336006873112]}, 'properties': {'OBJECTID': 79, 'SWR_TYPE': 'REPAIR OF STREET LIGHTING', 'SR_CATEGORY': 'STREET LIGHTING'}}, {'type': 'Feature', 'id': 80, 'geometry': {'type': 'Point', 'coordinates': [-82.98212215913381, 40.13336006873112]}, 'properties': {'OBJECTID

...

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-22+12%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-23+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 10, 'geometry': {'type': 'Point', 'coordinates': [-83.03761684973992, 39.98086490265935]}, 'properties': {'OBJECTID': 10, 'SWR_TYPE': 'WATER/MISCELLANEOUS', 'SR_CATEGORY': 'WATER, SEWERS & DRAINS'}}, {'type': 'Feature', 'id': 54, 'geometry': {'type': 'Point', 'coordinates': [-83.00343501893127, 39.96645703491427]}, 'properties': {'OBJECTID': 54, 'SWR_TYPE': 'REPAIR OF STREET LIGHTING', 'SR_CATEGORY': 'STREET LIGHTING'}}, {'type': 'Feature', 'id': 69, 'geometry': {'type': 'Point', 'coordinates': [-83.01201912934347, 40.03443591650526]}, 'properties': {'OBJECTID': 69, 'SWR_TYPE': 'REPLACE MISSIN

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-23+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-24+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 13, 'geometry': {'type': 'Point', 'coordinates': [-82.95078353392866, 40.08105826377182]}, 'properties': {'OBJECTID': 13, 'SWR_TYPE': 'FIRE HYDRANT REPAIRS', 'SR_CATEGORY': 'FIRE HYDRANT'}}, {'type': 'Feature', 'id': 90, 'geometry': {'type': 'Point', 'coordinates': [-83.00767136062345, 40.141960678016005]}, 'properties': {'OBJECTID': 90, 'SWR_TYPE': 'SEND LETTER TO PURCHASE ADDITIONAL REFUSE CONTAINER', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 91, 'geometry': {'type': 'Point', 'coordinates': [-83.00767136062345, 40.141960678016005]}, 'properties': {'OBJECTID': 91, 

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-24+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-25+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 71, 'geometry': {'type': 'Point', 'coordinates': [-83.11861934117913, 40.05481061383003]}, 'properties': {'OBJECTID': 71, 'SWR_TYPE': '64-GALLON MISSED SERVICE', 'SR_CATEGORY': 'RECYCLING / YARD WASTE'}}, {'type': 'Feature', 'id': 219, 'geometry': {'type': 'Point', 'coordinates': [-82.98603268277077, 40.071322397586975]}, 'properties': {'OBJECTID': 219, 'SWR_TYPE': 'STREET TREE MAINTENANCE', 'SR_CATEGORY': 'TREES'}}, {'type': 'Feature', 'id': 254, 'geometry': {'type': 'Point', 'coordinates': [-82.9631296333422, 39.90386678769116]}, 'properties': {'OBJECTID': 254, 'SWR_TYPE': 'CODE/TRASH AND DE

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-25+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-26+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 84, 'geometry': {'type': 'Point', 'coordinates': [-82.99426274197667, 40.01783257509952]}, 'properties': {'OBJECTID': 84, 'SWR_TYPE': '90-GALLON CONTAINER NEEDS SERVICED', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 85, 'geometry': {'type': 'Point', 'coordinates': [-82.99426274197667, 40.01783257509952]}, 'properties': {'OBJECTID': 85, 'SWR_TYPE': 'CODE/TRASH AND DEBRIS IN YARD', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 86, 'geometry': {'type': 'Point', 'coordinates': [-82.99426274197667, 40.01783257509952]}, 'properties': {'OBJECTID': 86,

http://maps2.columbus.gov/arcgis/rest/services/Applications/ServiceRequests/MapServer/0/query?where=DATE_OPEN+%3E%3D+TIMESTAMP+%272022-02-26+00%3A00%3A00%27+AND+DATE_OPEN+%3C+TIMESTAMP+%272022-02-27+00%3A00%3A00%27&f=geojson&outFields=OBJECTID%2CSWR_TYPE%2CSR_CATEGORY {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': [{'type': 'Feature', 'id': 4478, 'geometry': {'type': 'Point', 'coordinates': [-82.99502955019139, 39.93365987209683]}, 'properties': {'OBJECTID': 4478, 'SWR_TYPE': '64-GALLON MISSING/STOLEN', 'SR_CATEGORY': 'RECYCLING / YARD WASTE'}}, {'type': 'Feature', 'id': 3387, 'geometry': {'type': 'Point', 'coordinates': [-82.88809835498763, 39.91821364307715]}, 'properties': {'OBJECTID': 3387, 'SWR_TYPE': 'CODE/TRASH AND DEBRIS IN YARD', 'SR_CATEGORY': 'REFUSE / TRASH / LITTER'}}, {'type': 'Feature', 'id': 3745, 'geometry': {'type': 'Point', 'coordinates': [-83.06312933977883, 39.93792309009351]}, 'properties': {'OBJECTID': 3745

Read the features into a dataframe, do some cleanup, and write to a file

In [35]:
gdf = gpd.GeoDataFrame.from_features(features, crs="EPSG:3857")

# Drop duplicates present due to segment boundary conditions.
gdf.drop_duplicates(subset=["OBJECTID"], inplace=True)

# Convert timestamps to datetime objects
#gdf['DATE_OPEN'] = gdf['DATE_OPEN'].astype('datetime64[ms]')
#gdf['STATUS_DATE'] = gdf['STATUS_DATE'].astype('datetime64[ms]')

# Write the data into a GeoPackage (SQLite database)
tag = "{}-{}".format(startDateTimestamp.strftime("%Y%m%d"), endDateTimestamp.strftime("%Y%m%d"))
gdf.to_file("./columbus311-{}.gpkg".format(tag), driver="GPKG", layer=tag)

In [41]:
df = pd.DataFrame(gdf)
df.drop(columns=["geometry","OBJECTID"], inplace=True)
df.to_excel("30day.xlsx")

In [38]:
df

,geometry,OBJECTID,SWR_TYPE,SR_CATEGORY
0,POINT (-82.943 40.001),87,BED BUG ITEMS/SPECIAL COLLECTION,PUBLIC HEALTH
1,POINT (-82.983 40.127),115,REPLACE DAMAGED 90-GALLON REFUSE CONTAINER,REFUSE / TRASH / LITTER
2,POINT (-82.983 40.127),116,64-GALLON NEEDS REPLACED,RECYCLING / YARD WASTE
3,POINT (-83.072 40.128),70,90-GALLON CONTAINER NEEDS SERVICED,REFUSE / TRASH / LITTER
4,POINT (-83.077 39.950),191,RUMPKE/MISSED YARD WASTE,RECYCLING / YARD WASTE
...,...,...,...,...
9188,POINT (-83.063 39.938),3745,REQUEST TO ADD A 300-GALLON CONTAINER,REFUSE / TRASH / LITTER
9189,POINT (-82.979 40.001),8588,CODE/TRASH AND DEBRIS IN YARD,REFUSE / TRASH / LITTER
9190,POINT (-83.000 40.023),10759,RUMPKE/MISSED YARD WASTE,RECYCLING / YARD WASTE
9191,POINT (-83.074 39.941),9808,REPAIR 90-GALLON REFUSE CONTAINER,REFUSE / TRASH / LITTER
